* Este notebook es para ir desarrollando celda por celda las funciones de las consultas <br>
    a las cuales accederá el archivo main.py de la API para conseguir la información <br>
    Se cargarán los datos de los parquets que fueron creados en el note book <br>
    Data_Preparation_For_API

In [1]:
import pandas as pd

Se comienza con la funcion para el endpoint developer que es el que hace referencia a <br>
nuestro archivo parquet: contenido_free_items

In [2]:
df_developer = pd.read_parquet('data/contenido_free_items.parquet')

In [3]:
df_developer.head(10)

,price,release_year,developer,item_id
0,4.99,2018.0,Kotoshiro,761140
1,4.99,2018.0,Kotoshiro,761140
2,4.99,2018.0,Kotoshiro,761140
3,4.99,2018.0,Kotoshiro,761140
4,4.99,2018.0,Kotoshiro,761140
5,0.00,2018.0,Secret Level SRL,643980
6,0.00,2018.0,Secret Level SRL,643980
7,0.00,2018.0,Secret Level SRL,643980
8,0.00,2018.0,Secret Level SRL,643980
9,0.00,2017.0,Poolians.com,670290


In [4]:
# Para comenzar a crear la funcion se escoge un desarrollador al azar
dev = "Poolians.com"
# Filtramos por el desarrollador que se ingrese en la API
data_dev = df_developer[df_developer['developer'] == dev]


In [5]:
# Se la cantidad de items por año
items_per_year = data_dev.groupby('release_year')['item_id'].count().to_dict()
items_per_year

{2017.0: 5}

In [6]:
# Calculamos la cantidad de contenido free por año
free_count = data_dev[data_dev['price'] == 0.0].groupby('release_year')['item_id'].count().fillna(0).to_dict()
free_count

{2017.0: 5}

In [7]:
# Se calcula el porcentaje free
free_percentage = {year: (free_count.get(year, 0) / items_per_year.get(year, 1)) * 100 for year in items_per_year}
free_percentage

{2017.0: 100.0}

In [8]:
# Crear la estructura del diccionario de salida con cada año y sus valores asociados
developer_dict = {}
for year in items_per_year.keys():
    year_int = int(year)
    developer_dict[year_int] = {
            'Cantidad de Items': items_per_year.get(year, 0),
            '% Contenido Free': free_count.get(year, 0),
            'Porcentaje Free': free_percentage.get(year, 0)
        }
developer_dict

{2017: {'Cantidad de Items': 5,
  '% Contenido Free': 5,
  'Porcentaje Free': 100.0}}

Se continua ahora con la funcion para el endpoint userdata que es el que hace referencia a <br>
nuestro archivo parquet: user_data_price y tambien necesitaremos user_reviews_clean

In [9]:
df_reviews = pd.read_parquet('data/user_reviews_clean.parquet')

In [10]:
df_user_data = pd.read_parquet('data/user_data_price.parquet')

In [11]:
df_user_data

,items_count,user_id,price
0,58,--000--,1566.35
1,44,--ace--,348.58
2,23,--ionex--,169.86
3,68,-2SV-vuLB-Kg,942.80
4,149,-404PageNotFound-,3484.47
...,...,...,...
70907,5,zzonci,59.94
70908,61,zzoptimuszz,369.92
70909,13,zzydrax,209.88
70910,84,zzyfo,1382.17


In [37]:
# Se escoge un usuario para hacer pruebas
user_id = '76561197970982479'
# Se comienza filtrando por el usuario que se ingresa en la API
user = df_reviews[df_reviews['user_id'] == user_id]

In [38]:
# Se hace el calculo de la cantidad de dinero gastado para ese usuario
price_count = df_user_data[df_user_data['user_id']== user_id]['price'].iloc[0]
price_count

6427.36

In [39]:
 # Se obtiene la cantidad de items para ese usuario   
items_count = df_user_data[df_user_data['user_id']== user_id]['items_count'].iloc[0]
items_count

277

In [40]:
# Se busca el total de recomendaciones por ese usuario
user_recom_count = user['reviews_recommend'].sum()
user_recom_count

3

In [41]:
# Se calcula el total de reviews de todos los usuarios
reviews_total = len(df_reviews['user_id'].unique())
reviews_total

25132

In [42]:
# Ahora se puede calcular el porcentaje de ese usuario
user_percentage = (user_recom_count / reviews_total) * 100
user_percentage

0.011936972783702053

In [43]:
user_data_dic = {
        'cantidad_dinero': int(price_count),
        'porcentaje_recomendacion': round(float(user_percentage), 2),
        'total_items': int(items_count)
    }

user_data_dic

{'cantidad_dinero': 6427, 'porcentaje_recomendacion': 0.01, 'total_items': 277}

Ahora se trabaja con en endpoint UserForGenre que hace referencia al parquet <br>
user_for_genre

In [19]:
df_user_genre = pd.read_parquet('data/user_for_genre.parquet')

In [20]:
df_user_genre

,genres,user_id,release_year,playtime_hrs
0,Action,--000--,2009.0,88.816667
1,Action,--000--,2010.0,0.366667
2,Action,--000--,2011.0,108.700000
3,Action,--000--,2012.0,1822.433333
4,Action,--000--,2013.0,6.050000
...,...,...,...,...
3339195,Web Publishing,zachwgtv,2005.0,0.000000
3339196,Web Publishing,zachwgtv,2012.0,0.000000
3339197,Web Publishing,zepavil,2012.0,481.933333
3339198,Web Publishing,zepavil,2015.0,150.166667


In [21]:
# Se utiliza un genero de referencia
genre = 'Web Publishing'
# Se filtra para este genero 
one_genre_data = df_user_genre[df_user_genre['genres'] == genre]

In [22]:
# Se calcula el usuario con más horas
top_user_max = one_genre_data.groupby(['user_id'])['playtime_hrs'].sum().idxmax()
top_user_max

'Xyphien'

In [23]:
# Se calcula la suma de la cantidad total de horas
top_hrs_max = one_genre_data.loc[one_genre_data.set_index(['user_id']).index == top_user_max]['playtime_hrs'].sum()
top_hrs_max

2382.733333333333

In [24]:
# Crear una lista de diccionarios con años y horas jugadas para cada año
horas_por_año = []

available_years = one_genre_data.loc[one_genre_data['user_id'] == top_user_max, 'release_year'].unique()

for year in available_years:
    horas_año = one_genre_data[(one_genre_data['user_id'] == top_user_max) & (one_genre_data['release_year'] == year)]['playtime_hrs'].sum()
    if horas_año > 0:
        horas_por_año.append({'Año': int(year), 'Horas': int(horas_año)})

# Construir el diccionario final
resultado = {
    f"Usuario con más horas jugadas para Género {genre}": top_user_max,
    "Horas jugadas": horas_por_año
}

In [25]:
resultado

{'Usuario con más horas jugadas para Género Web Publishing': 'Xyphien',
 'Horas jugadas': [{'Año': 2005, 'Horas': 121},
  {'Año': 2012, 'Horas': 1077},
  {'Año': 2015, 'Horas': 1183}]}

Ahora se trabaja con en endpoint best_developer_year que hace referencia al parquet <br>
best_developer_year que ya fue construido con las columnas necesarias, ya tiene reviews <br>
igual a True y tambien tiene sentimiento de analisis positivo, por lo cual solo hay que armar <br>
el resultado de salida que es el diccionario

In [26]:
df_best_developer_year = pd.read_parquet('data/best_developer_year.parquet')

In [27]:
df_best_developer_year.head()

,sentiment_analysis,reviews_recommend,user_id,developer,release_year
0,2,True,76561197970982479,Valve,1998
1,2,True,76561197970982479,Arkane Studios,2002
2,2,True,76561197970982479,"2K Boston,2K Australia",2007
3,2,True,76561197970982479,"Epic Games, Inc.",1998
4,2,True,76561197970982479,Oddworld Inhabitants,1998


In [28]:
# Se escoge un año para pruebas
year_demo = 2002
# Se filtra el DataFrame para el año dado
year_data = df_best_developer_year[(df_best_developer_year['release_year'] == year_demo) & (df_best_developer_year['sentiment_analysis'] == 2)]

In [29]:
# Se obtienen los 3 principales desarrolladores con más juegos recomendados por usuarios
top_developers = year_data.groupby('developer').size().nlargest(3)
top_developers

developer
Io-Interactive A/S       1227
Bethesda Game Studios     848
Rockstar Games            716
dtype: int64

In [30]:
# Se construye el diccionario con el resultado
resultado = [{"Puesto 1": top_developers.index[0]}, {"Puesto 2": top_developers.index[1]}, {"Puesto 3": top_developers.index[2]}]
resultado

[{'Puesto 1': 'Io-Interactive A/S'},
 {'Puesto 2': 'Bethesda Game Studios'},
 {'Puesto 3': 'Rockstar Games'}]

In [31]:
resultado = {
    "Puesto 1": top_developers.index[0],
    "Puesto 2": top_developers.index[1],
    "Puesto 3": top_developers.index[2]
}
resultado

{'Puesto 1': 'Io-Interactive A/S',
 'Puesto 2': 'Bethesda Game Studios',
 'Puesto 3': 'Rockstar Games'}

Finalmente, se va a trabajar con el endpoint developer_reviews_analysis que tiene relacionado al parquet <br>
dev_rev_analysis.parquet que ya tiene disponibles dos columnas: desarollador y analisis de sentimiento.

In [32]:
df_dev_analysis = pd.read_parquet('data/dev_rev_analysis.parquet')

In [33]:
df_dev_analysis

,developer,sentiment_analysis
0,Valve,2
1,Arkane Studios,2
2,"2K Boston,2K Australia",2
3,"Epic Games, Inc.",2
4,Oddworld Inhabitants,2
...,...,...
3167896,Secret Exit Ltd.,2
3167897,Bohemia Interactive,2
3167898,Ubisoft Montreal,2
3167899,Avalanche Studios,2


In [34]:
# Se selecciona un desarrollador para pruebas
dev_demo = 'Valve'
# Se filtra el DataFrame por el desarrollador dado
developer_data = df_dev_analysis[df_dev_analysis['developer'] == dev_demo]

In [35]:
# Se obtiene el recuento de análisis de sentimiento por desarrollador
sentiment_counts = developer_data['sentiment_analysis'].value_counts()
sentiment_counts

sentiment_analysis
2    76943
1    75683
0    31810
Name: count, dtype: int64

In [36]:
# Se crea el diccionario de salida
resultado = {dev_demo: [f"Negative = {sentiment_counts.get(0, 0)}", f"Positive = {sentiment_counts.get(2, 0)}"]}
resultado

{'Valve': ['Negative = 31810', 'Positive = 76943']}

Ahora que ya tenemos los resultados en este notebook, se va a construir una funcion llamada "helper.py" <br>
Ahí se van a poner todas las funciones que serán de utilidad para devolver la informacion en la API.
Con este notebook finalizado, se proecede a desarrollar el archivo "helper.py"